In [53]:
import numpy as np
import cv2
import random
import os

## image is an np array of the image


def rotate_image(image, angle):
    height, width = image.shape[:2]
    rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))
    return rotated_image


def flip_image(image):
    flip_variations = [
        (False, False),  # No flip
        (True, False),  # Horizontal flip
        (False, True),  # Vertical flip
        (True, True),  # Both flips
    ]

    random.shuffle(flip_variations)
    augmented_images = []

    for flip_horizontal, flip_vertical in flip_variations[:4]:
        if flip_horizontal and flip_vertical:
            augmented_images.append(cv2.flip(image, -1))  # horizontal and vertical
        elif flip_horizontal:
            augmented_images.append(cv2.flip(image, 1))  # horizontal
        elif flip_vertical:
            augmented_images.append(cv2.flip(image, 0))  # vertical
        else:
            augmented_images.append(image)  # No flip

    return augmented_images


def shear_image(image, shear_range):
    height, width = image.shape[:2]
    shear_value = random.uniform(-shear_range, shear_range)

    if shear_value < 0:
        shear_matrix = np.array([[1, -shear_value, 0], [0, 1, 0]])
    else:
        shear_matrix = np.array([[1, shear_value, 0], [0, 1, 0]])

    sheared_image = cv2.warpAffine(image, shear_matrix, (width, height))
    return sheared_image


def translate_image(image, translate_range):
    height, width = image.shape[:2]

    max_shift_x = int(width * 0.1)
    max_shift_y = int(height * 0.1)

    translate_x = random.randint(-max_shift_x, max_shift_x)
    translate_y = random.randint(-max_shift_y, max_shift_y)

    translation_matrix = np.array(
        [[1, 0, translate_x], [0, 1, translate_y]], dtype=np.float32
    )

    translated_image = cv2.warpAffine(image, translation_matrix, (width, height))

    return translated_image


def adjust_brightness(image, brightness_range):
    brightness_factor = 1.0 + random.uniform(-brightness_range, brightness_range)
    adjusted_image = np.clip(image * brightness_factor, 0.25, 255).astype(np.uint8)
    return adjusted_image

In [57]:
## image parameter is an np array of an image
## flag 1: includes brightness, 0 or any other value exludes it


def augmented_fn2(image, flag):
    augmented_images = []

    if flag == 1:
        for _ in range(4):
            angle = random.uniform(-35, 35)
            image_r = rotate_image(image, angle)
            augmented_images.append(image_r)

        image_f = flip_image(image)
        augmented_images.extend(image_f)

        for _ in range(4):
            image_s = shear_image(image, shear_range=0.15)
            augmented_images.append(image_s)

        for _ in range(4):
            image_t = translate_image(image, translate_range=0.1)
            augmented_images.append(image_t)

        for _ in range(4):
            image_b = adjust_brightness(image, brightness_range=0.25)
            augmented_images.append(image_b)
    else:
        for _ in range(4):
            angle = random.uniform(-35, 35)
            image_r = rotate_image(image, angle)
            augmented_images.append(image_r)

        image_f = flip_image(image)
        augmented_images.extend(image_f)

        for _ in range(4):
            image_s = shear_image(image, shear_range=0.15)
            augmented_images.append(image_s)

        for _ in range(4):
            image_t = translate_image(image, translate_range=0.1)
            augmented_images.append(image_t)

    return np.array(augmented_images)

Examples:

In [60]:
image_o = cv2.imread("007-0035-000.jpg")
augmented_images = augmented_fn2(image_o, 1)

In [59]:
# save example
image_name = "007-0035-000"


def save_augmentation(image, output_folder, flag):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    augmented_images = augmented_fn2(image, flag)

    for idx, augmented_image in enumerate(augmented_images, 1):
        cv2.imwrite(
            os.path.join(output_folder, f"{image_name}_{idx}.jpg"), augmented_image
        )


image_o = cv2.imread("007-0035-000.jpg")
save_augmentation(image_o, "Augmented_images4/", flag=1)